In [26]:
import os
import math
import time
import copy
import random
import numpy as np
import torch.nn as nn
import torch.autograd
from skimage import io
from torch import optim
import torch.nn.functional as F
from tensorboardX import SummaryWriter
from torch.utils.data import DataLoader

working_path = os.path.dirname(os.path.dirname(os.getcwd()))

result_path = os.path.join(working_path, 'SCanNet/ResultsAnnotation/Figures/')

working_path = os.path.join(working_path, 'SCanNet')
import sys
sys.path.append(working_path)
print('working_path:', working_path)

from utils.loss import CrossEntropyLoss2d, weighted_BCE_logits, ChangeSimilarity
from utils.utils import accuracy, SCDD_eval_all, AverageMeter
from datasets import RS_ST as RS
#from models.TED import TED as Net
from models.SCanNet import SCanNet as Net

working_path: /home/buddhiw/ChangeDetection/CDSCanNet/Lovasz_Dice_Losses/SCanNet


In [27]:
NET_NAME = 'SCanNet_Ours'
DATA_NAME = 'ST'

CHECKPOINTDIR = os.path.join(working_path, 'checkpoints/ST/SCanNet_psd_49e_mIoU73.09_Sek23.30_Fscd63.33_OA87.30.pth')

In [28]:
net = Net(3, num_classes=RS.num_classes).cuda()
net.load_state_dict(torch.load(CHECKPOINTDIR), strict=False)
net.eval()

/home/buddhiw/miniconda3/envs/Mamba/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/buddhiw/miniconda3/envs/Mamba/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


SCanNet(
  (FCN): FCN(
    (layer0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, 

In [29]:
DATA_DIR = os.path.dirname(os.path.dirname(os.path.dirname(working_path)))
root = os.path.join(DATA_DIR, 'Datasets', 'SECOND')
print('Loading dataset from %s' % root)

Loading dataset from /home/buddhiw/ChangeDetection/Datasets/SECOND


In [30]:
from datasets.RS_ST import *
image_1 = io.imread(os.path.join(root, 'train', 'T1', '01437.png'))
image_1 = normalize_image(image_1)

image_2 = io.imread(os.path.join(root, 'train', 'T2', '01437.png'))
image_2 = normalize_image(image_2)

label_1 = io.imread(os.path.join(root, 'train', 'GT_T1', '01437.png'))
label_2 = io.imread(os.path.join(root, 'train', 'GT_T2', '01437.png'))

from torchvision.transforms import functional as TF
import torch.nn.functional as F

image_1, image_2, label_1, label_2 = TF.to_tensor(image_1), TF.to_tensor(image_2), TF.to_tensor(label_1), TF.to_tensor(label_2)

image_1 = image_1.unsqueeze(0).cuda()
image_2 = image_2.unsqueeze(0).cuda()
label_1 = label_1.unsqueeze(0).cuda()
label_2 = label_2.unsqueeze(0).cuda()


In [31]:
imgs_A, imgs_B, labels_A, labels_B, imgs_id = image_1, image_2, label_1, label_2, 1437
if True:
    imgs_A = imgs_A.cuda().float()
    imgs_B = imgs_B.cuda().float()
    labels_A = labels_A.cuda().long()
    labels_B = labels_B.cuda().long()

with torch.no_grad():
    out_change, outputs_A, outputs_B = net(imgs_A, imgs_B)

labels_A = labels_A.cpu().detach().numpy()
labels_B = labels_B.cpu().detach().numpy()
outputs_A = outputs_A.cpu().detach()
outputs_B = outputs_B.cpu().detach()
change_mask = F.sigmoid(out_change).cpu().detach() > 0.5
preds_A = torch.argmax(outputs_A, dim=1)
preds_B = torch.argmax(outputs_B, dim=1)
preds_A = (preds_A * change_mask.squeeze().long()).numpy()
preds_B = (preds_B * change_mask.squeeze().long()).numpy() 
pred_A_color = RS.Index2Color(preds_A[0])
pred_B_color = RS.Index2Color(preds_B[0])
io.imsave(os.path.join(result_path, NET_NAME + f'{imgs_id}_A.png'), pred_A_color)
io.imsave(os.path.join(result_path, NET_NAME + f'{imgs_id}_B.png'), pred_B_color)
print('Prediction saved!')


Prediction saved!
